*The codes were developed on Windows 10, and were not tested on other machines. Anaconda 5.2.0 is chosen as a Python interpreter.*

This notebook introduces the concept and methodology of implementing Bootstrap Simulation.

# Bootstrap Simulation

> **Bootstrap Simulation** is a technique to understand the impact of risk and uncertainty in 
finance, engineering, supply chain, and virtually anything. A Monte Carlo simulator helps
one to visualize most or all of the potential outcomes to have a better idea regaring the
risk of a decision.

Based on historical data, expertise in the field, or past experience, you can draw an estimate of a population. While this estimate is useful for developing a model, it contains some inherent uncertainty and risk, because it's an estimate of an unknown value. In a financial market, you might know the distribution of possible values through the mean and standard deviation of returns. By using a range of possible values, instead of a single guess, you can create a more realistic picture of what might happen in the future. 